In [2]:
import numpy as np
import pandas as pd
import scipy as sci
import os, glob
'''
Read the station infomation and event information to rotate it from ZNE to ZRT coordinates
Reference: Plešinger, A., M. Hellweg and D. Seidl, Interactive high-resolution polarization analysis of broadband seismograms.
J. Geophysics, 59, p. 129-139, 1986.

'''
def read_cwb_pandas(filepath,filename):
    import pandas as pd
    from pyproj import Geod
    '''
    read the cwb format using pandas and trigger output
    * Input file_path and file_name
    * Output: Baz, dataframe for ZNE
    '''
    # Capture the station coordinate from cwb header
    # # Capture the station coordinate from cwb header
    file = open(filepath+filename, 'r')
    Lines = file.readlines()[0:16]
    file.close()
    # station codes
    sta_nm=Lines[0].split(" ")[-1].split("\n")[0]
    sta_lat=float(Lines[1].split(" ")[-1])
    sta_lon=float(Lines[2].split(" ")[-1])
    # event codes
    evt_lat=float(Lines[3].split(" ")[-1])
    evt_lon=float(Lines[4].split(" ")[-1])
    # Calculate the azimuthal and distance
    g = Geod(ellps='WGS84')
    az,baz,dist = g.inv(sta_lon,sta_lat,evt_lon,evt_lat)
    # Now get the data
    data = pd.read_csv(filepath+filename,
                       skiprows=16,
                       delim_whitespace=True,
                       names=("time","Z","N","E"),
                      )
    return data,baz,Lines
    # 
def rotate_data(data,baz):
    import numpy as numpy
    import pandas as pd
    '''
    rotate the seismogram with input backazimuthal angle
    replace the N and E colunms by R and T
    '''
    data["R"]=-data["N"]*np.cos(np.radians(baz)) - data["E"]*np.sin(np.radians(baz));
    data["T"]=data["N"]*np.sin(np.radians(baz)) - data["E"]*np.cos(np.radians(baz));
    maxR=data["R"].max();
    minR=data["R"].min();
    maxT=data["R"].max();
    minT=data["R"].min();
    new_data = data.drop(columns=['N', 'E'])
    return new_data,maxR,minR,maxT,minT
    #
def write_csv(filepath,filename,header,data,maxR,minR,maxT,minT):
    f = open(filepath+"/"+filename,"w")
    for i in range(12):
        f.write(header[i])
    f.write('%AmplitudeMAX. R:{:7.4f}~{:7.4f}\n'.format(maxR, minR))
    f.write('%AmplitudeMAX. T:{:7.4f}~{:7.4f}\n'.format(maxT, minT))
    f.write("%DataSequence: Time U(+); R(+), T(+)\n")
    f.write('%Data: 4F10.3\n')
    #
    for i in range(len(data["time"])):
        f.write('%10.3f%10.3f%10.3f%10.3f\n'%(data.iloc[i]["time"],data.iloc[i]["Z"],data.iloc[i]["R"],data.iloc[i]["T"]))
    f.close()
    return
# ============================================================================================================
indir="tsmip_record_April3"
#
outdir=f"{indir}_ZRT"
# ============================================================================================================
#
if not os.path.isdir(outdir):
        os.makedirs(outdir)
# Now check the input directory
for fullfile in glob.glob(f"{indir}/*.cwb"):
    print(fullfile)
    filepath=""
    for i in [len(fullfile.split("/"))-2]:
        filepath=filepath+fullfile.split("/")[i]+"/"
    filename=fullfile.split("/")[-1]
    # print(filepath,filename)
    # 
    data,baz,headers = read_cwb_pandas(filepath,filename)
    new_data,maxR,minR,maxT,minT = rotate_data(data,baz)
    write_csv(outdir,filename,headers,data,maxR,minR,maxT,minT)
print("done!")


tsmip_record_April3/A054_20220917134117_100.0hz.cwb
tsmip_record_April3/A061_20220917134117_100.0hz.cwb
tsmip_record_April3/A049_20220917134117_100.0hz.cwb
tsmip_record_April3/A024_20240403001125_100.0hz.cwb
tsmip_record_April3/A077_20240403001125_100.0hz.cwb
tsmip_record_April3/A004_20240402235811_100.0hz.cwb
tsmip_record_April3/A043_20240403001125_100.0hz.cwb
tsmip_record_April3/A076_20220918064413_100.0hz.cwb
tsmip_record_April3/A082_20240403001125_100.0hz.cwb
tsmip_record_April3/A032_20220917134117_100.0hz.cwb
tsmip_record_April3/A128_20240402235811_100.0hz.cwb
tsmip_record_April3/A061_20240403001125_100.0hz.cwb
tsmip_record_April3/A082_20240402235811_100.0hz.cwb
tsmip_record_April3/A124_20240403001125_100.0hz.cwb
tsmip_record_April3/A134_20220918064413_100.0hz.cwb
tsmip_record_April3/A057_20220917134117_100.0hz.cwb
tsmip_record_April3/A084_20220918064413_100.0hz.cwb
tsmip_record_April3/A002_20220918064413_100.0hz.cwb
tsmip_record_April3/A107_20220917134117_100.0hz.cwb
tsmip_record